In [ ]:
from Data.Drosophilla.FlyDataMod import FlyDataModule
from Models.Transformer import PositionalEncoding, TransformerModule
from pytorch_lightning.callbacks import EarlyStopping
from IPython.core.debugger import set_trace
from matplotlib import gridspec

from Models import BiLSTM as bi
from Models import Transformer as tr
from Models import LinearRegression as ln

import glob

import pytorch_lightning as pl
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from Utils import evaluations as ev


In [ ]:

csv_files = ["Experiments/Table_1_Transformer_Tunning_Gamma/Sup_1_Results.csv",
             "Experiments/Table_2_Transformer_Tunning_Insulation/Sup_2_Results.csv",
             "Experiments/Table_3_Transformer_Tunning_Directionality/Sup_3_Results.csv",
             'Experiments/Table_7_LSTM_Tunning_Gamma/Sup_7_Results.csv',
             'Experiments/Table_8_LSTM_Tunning_Insulation/Sup_8_Results.csv',
             'Experiments/Table_9_LSTM_Tunning_Directionality/Sup_9_Results.csv']


gs = gridspec.GridSpec(2,3, wspace=0, hspace=0)
labels= ['converge', 'failed']
fig, ax = plt.subplots(2,3, figsize=(4,2))
for i in range(0,2):
    for j in range(0,3):
        x   = pd.read_csv(csv_files[i*3+j], skiprows=1)
        yes = np.sum(x['val_pcc'].to_numpy()>=.1)
        no  = np.sum(x['val_pcc'].to_numpy()<.1)
        ax[i,j] = plt.subplot(gs[i,j])
        ax[i,j].pie([yes,no],
            autopct='%1.1f%%',
         colors=['mediumpurple' , 'darkviolet'])
plt.show()

fig, ax = plt.subplots(figsize=(5,5))
ax.plot([1], label='converged', color='mediumpurple')
ax.plot([1], label='no converge', color='darkviolet')
plt.legend(prop={'size':16})
plt.show()

In [ ]:

model_names      = ['linear','ridge','lasso','bilstm','transformer']

model_pacs       = [ln.LinearRegressionModule,
                  ln.LinearRegressionModule,
                  ln.LinearRegressionModule,
                  bi.BiLSTMModule,
                  tr.TransformerModule]
                  
ra               = [0,10,20,0,0]
rb               = [10,20,30,50,50]
mse_idx          = [5,5,5,8,8]

def benchmark(label_type, label_val, model_folders, exp_indx):
    model_weights    = {}
    models           = {}

    
    second_folders   =    [str(label_type)+"_"+str(label_val)+"_RFalse_LFalse/", 
                      str(label_type)+"_"+str(label_val)+"_RTrue_LFalse/",
                      str(label_type)+"_"+str(label_val)+"_RFalse_LTrue/",
                      "",
                      ""]
    
    dm = FlyDataModule(cell_line="S2",
                  data_win_radius=5,
                  batch_size=1,
                  label_type=label_type,
                  label_val=label_val)
    dm.setup()
    for i, (name, folder) in enumerate(zip(model_names, model_folders)):
        model_doc     = pd.read_csv("Experiments/"+str(folder)+"/Sup_"+str(exp_idx[i])+"_Results.csv",
                               skiprows=1)
        model_vec     = model_doc.iloc[ra[i]:rb[i],mse_idx[i]]
        hparm_num     = np.argmax(model_vec)
        model_str     = "".join(["Experiments/",
                            folder,
                             second_folders[i],
                             "optuna/version_",
                             str(hparm_num),
                             "/checkpoints/*"
                            ])
        print(model_str)
        model_weights[name]  = glob.glob(model_str)[0]

    fig, ax = plt.subplots(1, figsize=(8,4))
    for i, name in enumerate(model_names):
        models[name] = model_pacs[i].load_from_checkpoint(model_weights[name]).to("cuda:0")
        label, output = ev.getModelPredictions(models[name], dm, 'test')
        ax.plot(output[600:700], label=name)
    
    ax.plot(label[600:700],
       label=label_type,
       color="black",
       linestyle='dashed')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    plt.legend()
    ax.set_title(label_type)
    plt.show()
    
    data = np.zeros((5,5))
    for i, name in enumerate(model_names):
        data[i,:] = list(ev.getModelMetrics(models[name], dm, 'test', clip_type=label_type).values())
        ar_val    = np.array(data)
        ar_col    = np.expand_dims(np.array(['mse','mae','r2','pcc','spc']),0)
        ar_row    = np.expand_dims(np.array(['metric','linear','lasso','ridge','bilstm','transformer']),1)
        csv_data  = np.vstack((ar_col, ar_val))
        csv_data  = np.hstack((ar_row, csv_data))
        pd.DataFrame(csv_data).to_csv("Experiments/BenchMark_"+str(label_type)+".csv")


In [ ]:

# gamma
label_type = "gamma"
label_val  = 0
model_folders    = ['Table_4_Linear_Regression_Tunning/',
                    'Table_4_Linear_Regression_Tunning/',
                    'Table_4_Linear_Regression_Tunning/',
                    'Table_7_LSTM_Tunning_Gamma/',
                    'Table_1_Transformer_Tunning_Gamma/']
exp_idx          = [4,4,4,7,1]

benchmark(label_type,
         label_val,
         model_folders,
         exp_idx)



#insulation
label_type    = "insulation"
label_val     = 3
model_folders = ["Table_5_Linear_Regression_Tunning/",
                 "Table_5_Linear_Regression_Tunning/",
                 "Table_5_Linear_Regression_Tunning/",
                "Table_8_LSTM_Tunning_Insulation/",
                "Table_2_Transformer_Tunning_Insulation/"]
exp_idx       = [5,5,5,8,2]

benchmark(label_type,
        label_val,
        model_folders,
         exp_idx)



#insulation
label_type    = "directionality"
label_val     = 10
model_folders = ["Table_6_Linear_Regression_Tunning/",
                 "Table_6_Linear_Regression_Tunning/",
                 "Table_6_Linear_Regression_Tunning/",
                "Table_9_LSTM_Tunning_Directionality/",
                "Table_3_Transformer_Tunning_Directionality/"]
exp_idx       = [6,6,6,9,3]

benchmark(label_type,
        label_val,
        model_folders,
         exp_idx)